# 03 — Safety Constraint Module

Deep dive into the safety system — the paper's key contribution.

Demonstrates:
1. Battery-first plan validation
2. Real-time force monitoring
3. How the safety module intervenes during execution
4. Generating figures for the paper

In [ ]:
import sys
sys.path.insert(0, '..')

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 5)

## 3.1 — Plan Validation: Battery-First Rule

In [ ]:
from safedisassemble.models.safety.constraint_checker import (
    SafetyConstraintModule, SafetyLevel, SafetyAction
)

safety = SafetyConstraintModule()

# SAFE plan: battery before internals
safe_plan = [
    {'component': 'screw_1', 'component_type': 'screw', 'step_id': 1},
    {'component': 'screw_2', 'component_type': 'screw', 'step_id': 2},
    {'component': 'back_panel', 'component_type': 'panel', 'step_id': 3},
    {'component': 'battery', 'component_type': 'battery', 'step_id': 4},
    {'component': 'ram', 'component_type': 'ram', 'step_id': 5},
    {'component': 'ssd', 'component_type': 'ssd', 'step_id': 6},
]

result = safety.validate_plan(safe_plan)
print(f"Safe plan:   {result.level.value:10s} | {result.reason}")

# UNSAFE plan: RAM before battery
safety.reset()
unsafe_plan = [
    {'component': 'screw_1', 'component_type': 'screw', 'step_id': 1},
    {'component': 'back_panel', 'component_type': 'panel', 'step_id': 2},
    {'component': 'ram', 'component_type': 'ram', 'step_id': 3},  # BEFORE battery!
    {'component': 'battery', 'component_type': 'battery', 'step_id': 4},
    {'component': 'ssd', 'component_type': 'ssd', 'step_id': 5},
]

result = safety.validate_plan(unsafe_plan)
print(f"Unsafe plan: {result.level.value:10s} | {result.reason}")
print(f"  Action: {result.action.value}")
print(f"  Violations: {result.details}")

## 3.2 — Runtime Force Monitoring Simulation

In [ ]:
from safedisassemble.models.safety.constraint_checker import ForceMonitor

monitor = ForceMonitor(warning_threshold_fraction=0.7)
monitor.register_zone(
    name='battery_zone',
    position=np.array([0.44, 0.0, 0.44]),
    radius=0.06,
    max_force=15.0,
)

# Simulate a trajectory approaching then pressing on the battery zone
n_steps = 100
ee_trajectory = np.zeros((n_steps, 3))
force_trajectory = np.zeros(n_steps)
safety_levels = []

for t in range(n_steps):
    # Move from far to near, then apply increasing force
    progress = t / n_steps
    
    if progress < 0.3:
        # Approach phase
        ee_pos = np.array([0.2 + progress, 0.0, 0.6 - progress * 0.5])
        force = np.array([0.0, 0.0, 0.5])
    elif progress < 0.6:
        # Near battery, moderate force
        ee_pos = np.array([0.44, 0.0, 0.45])
        force = np.array([0.0, 0.0, 3.0 + (progress - 0.3) * 30])
    else:
        # Dangerous: high force on battery zone
        ee_pos = np.array([0.44, 0.0, 0.44])
        force = np.array([0.0, 0.0, 12.0 + (progress - 0.6) * 20])
    
    ee_trajectory[t] = ee_pos
    force_trajectory[t] = np.linalg.norm(force)
    
    assessments = monitor.update(ee_pos, force)
    if assessments:
        safety_levels.append((t, assessments[0].level.value))

# Plot the scenario
fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

# Force over time
axes[0].plot(force_trajectory, 'b-', linewidth=2, label='Applied force')
axes[0].axhline(y=15.0, color='red', linestyle='--', linewidth=2, label='Puncture threshold (15N)')
axes[0].axhline(y=15.0 * 0.7, color='orange', linestyle='--', linewidth=1, label='Warning threshold (70%)')
axes[0].fill_between(range(n_steps), 0, force_trajectory,
                      where=force_trajectory > 15.0, color='red', alpha=0.2)
axes[0].fill_between(range(n_steps), 0, force_trajectory,
                      where=(force_trajectory > 10.5) & (force_trajectory <= 15.0),
                      color='orange', alpha=0.2)
axes[0].set_ylabel('Force (N)', fontsize=12)
axes[0].legend(fontsize=11)
axes[0].set_title('Safety Monitor: Force on Battery Zone', fontsize=14)

# Distance to zone
distances = np.linalg.norm(ee_trajectory - np.array([0.44, 0.0, 0.44]), axis=1)
axes[1].plot(distances, 'g-', linewidth=2)
axes[1].axhline(y=0.06, color='red', linestyle=':', label='Zone radius')
axes[1].set_ylabel('Distance to zone (m)', fontsize=12)
axes[1].set_xlabel('Timestep', fontsize=12)
axes[1].legend(fontsize=11)

# Mark safety events
for t, level in safety_levels:
    color = {'critical': 'red', 'warning': 'orange', 'caution': 'yellow'}.get(level, 'gray')
    axes[0].axvline(x=t, color=color, alpha=0.1)

plt.tight_layout()
plt.savefig('../renders/safety_force_monitoring.png', dpi=150, bbox_inches='tight')
plt.show()
print(f"\nSafety events: {len(safety_levels)}")
print(f"Critical events: {sum(1 for _, l in safety_levels if l == 'critical')}")

## 3.3 — Full Safety Workflow

In [ ]:
from safedisassemble.sim.device_registry import get_device

safety = SafetyConstraintModule()
device_spec = get_device('laptop_v1')
safety.setup_from_device_spec(device_spec)

# Step through a disassembly sequence
steps = [
    ('screw_1', 'screw'),
    ('screw_2', 'screw'),
    ('screw_3', 'screw'),
    ('screw_4', 'screw'),
    ('screw_5', 'screw'),
    ('back_panel', 'panel'),
    ('battery', 'battery'),  # Safety-critical
    ('ram_module', 'ram'),
    ('ssd_module', 'ssd'),
]

print("Stepping through safe disassembly sequence:\n")
for comp_name, comp_type in steps:
    # Pre-action check
    check = safety.check_pre_action(comp_name)
    status = '✅' if check.is_safe else '❌'
    print(f"  {status} {comp_name:15s} -> {check.level.value:10s} | {check.reason}")
    
    # Execute removal
    safety.notify_removal(comp_name, comp_type)

print(f"\nSafety summary:")
summary = safety.get_safety_summary()
for k, v in summary.items():
    if k != 'zone_states':
        print(f"  {k}: {v}")